In [77]:
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
import optuna
from optuna.samplers import TPESampler

In [68]:
dataset = pd.read_csv('../db/AmesHousing.csv', sep='\t')
localCopy=dataset.copy(deep=True)

In [69]:
#SEPARATION OF CATEGORICAL AND NUMERICAL FEATURES

#AmesExplanations.txt indicates a separation of the data in 4 types:
# Continuous - numerical
# Nominal - categorical
# Ordinal - usually categorical, numerical if order matters
# Discrete - usually numerical, categorical if it's an ID

cat_features = ['MS SubClass','MS Zoning','Street','Alley',
                'Lot Shape','Land Contour','Utilities','Lot Config',
                'Land Slope','Neighborhood','Condition 1','Condition 2',
                'Bldg Type','House Style','Roof Style','Roof Matl',
                'Exterior 1st','Exterior 2nd','Mas Vnr Type','Foundation',
                'Heating','Central Air','Electrical','Garage Type',
                'Paved Drive','Misc Feature','Sale Condition']

num_features = ['Lot Frontage','Lot Area','Overall Qual','Overall Cond',
                'Year Built','Year Remod/Add','Mas Vnr Area','Exter Qual',
                'Exter Cond','Bsmt Qual','Bsmt Cond','Bsmt Exposure',
                'BsmtFin Type 1', 'BsmtFin SF 1','BsmtFin Type 2','BsmtFin SF 2',
                'Bsmt Unf SF','Total Bsmt SF','Heating QC','1st Flr SF','2nd Flr SF',
                'Low Qual Fin SF','Gr Liv Area','Bsmt Full Bath','Bsmt Half Bath',
                'Full Bath','Half Bath','Bedroom','Kitchen','Kitchen Qual','TotRmsAbvGrd',
                'Functional','Fireplaces','Fireplace Qu',
                'Garage Yr Blt','Garage Finish','Garage Cars','Garage Area','Garage Qual',
                'Garage Cond','Wood Deck SF','Open Porch SF','Enclosed Porch','3-Ssn Porch',
                'Screen Porch','Pool Area','Pool QC','Fence','Misc Val','Mo Sold','Yr Sold',
                'Sale Type']


# Features that describe a grade (ex. Quality: 1-10 or Poor/Average/Excellent) will be treated as
# numeric values. There are string values as well; these will be mapped to numbers so that the model
# understands them

# to be mapped to numbers: Exter Qual, Exter Cond, Bsmt Qual, Bsmt Cond, Bsmt Exposure,
# BsmtFin Type 1, BsmtFinType 2, HeatingQC, KitchenQual, Functional, FireplaceQu, 
# Garage Finish, Garage Qual, Garage Cond, Pool QC, Fence, Sale Type

ordinal_maps = {'Exter Qual': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Exter Cond': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Bsmt Qual': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Bsmt Cond': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Bsmt Exposure': {'NA':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4},
                'BsmtFin Type 1': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
                'BsmtFin Type 2': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
                'Heating QC': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Kitchen Qual': {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Functional': {'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8},
                'Fireplace Qu': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Garage Finish': {'NA':0, 'Unf':1, 'RFn':2, 'Fin':3},
                'Garage Qual': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Garage Cond': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
                'Pool QC': {'NA':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4},
                'Fence': {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4},
                'Sale Type': {'Oth':1, 'ConLD':2, 'ConLI':3, 'ConLw':4, 'Con':5, 'COD':6, 'New':7, 'VWD':8, 'CWD':9, 'WD':10}
                }

# apply the mappings to the strings
for col,mapping in ordinal_maps.items():
    if col in localCopy.columns:
        localCopy[col] = localCopy[col].map(mapping)

In [70]:
#FEATURE ENGINEERING: Deletion

#dropping the PID , Order Column
# Yr Sold, Sale Price, Year Built, Total Bsmt SF, Gr Liv Area because they have a high correlation with 
# other features, we are gonna keep them and delete the others
localCopy.drop(columns=['PID','Order','Overall Qual', 'Garage Yr Blt', '1st Flr SF','TotRms AbvGrd', 'Garage Cars'],inplace=True)
localCopy.drop(index=[1341,1497,1577,2236],inplace=True)

#dropping columns with many null values
for column in localCopy:
    if localCopy[column].isna().sum()>2000:
        localCopy.drop(column, axis=1, inplace=True)

#After all deletions, remove the deleted features in the 2 cat/num separation arrays as well
cat_features = [col for col in cat_features if col in localCopy.columns]
num_features = [col for col in num_features if col in localCopy.columns]
#Also do the same for the ordinal_maps
ordinal_maps = {col: mapping for col,mapping in ordinal_maps.items() if col in localCopy.columns}

In [71]:
#Separate the target variable from the rest
indexOfTV = localCopy.columns.get_loc("SalePrice")
X = localCopy.iloc[:,:indexOfTV]  #all rows and all columns except for the TV column
Y = localCopy.iloc[:,indexOfTV]  #column of the Target Variable with all the rows (":")

# compute the quartiles 
# step 1: copy/sort the data
Y_copy = Y.copy(deep=True).sort_values(ascending=True)
length = len(Y_copy)
# a ranges vector that defines the classes: e.g. for the class 1 we must look up to be between the 
# ranges[0] and ranges[1] => for each class 'i' the range is defined by (ranges[i-1], ranges[i])
min = np.min(np.array(Y)); max = np.max(np.array(Y))
ranges = [min, -1, -1, -1, max]

# length is even then:
# Q2 = mean(length/2, length/2-1)
Q2 = np.mean(np.array(Y_copy.iloc[(length//2)-1:length//2+1]))

# split the Series and compute the Q1, Q3
cond = Y_copy <= Q2
half1, half2 = Y_copy[cond], Y_copy[~cond]

Q1 = np.median(np.array(half1))
Q3 = np.median(np.array(half2))

ranges[1] = Q1
ranges[2] = Q2 
ranges[3] = Q3

# change the target varaible to a categorical format
classes = ["cheap", "affordable", "expensive", "billionaire type of shit"]
cat_y: list[str] = []
for y in Y:
    if y <= ranges[1]:
        cat_y.append(classes[0])
    elif y <= ranges[2]:
        cat_y.append(classes[1])
    elif y <= ranges[3]:
        cat_y.append(classes[2])
    else:
        cat_y.append(classes[3])

Y = pd.Series(cat_y)

In [72]:
#FEATURE ENGINEERING: Scaling

#Scalam doar features numerice
scaler = MinMaxScaler()
scaled_data = X.copy(deep=True)
scaled_data[num_features] = scaler.fit_transform(X[num_features])

X = scaled_data

In [73]:
# One-Hot Encoding for the categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

encodedData = encoder.fit_transform(X[cat_features])
encodedDataFrame = pd.DataFrame(encodedData, columns=encoder.get_feature_names_out(cat_features))

X = pd.concat([X, encodedDataFrame], axis=1)
X = X.drop(cat_features, axis=1)
X.drop(index=[1341,1497,1577,2236], axis=0, inplace=True)


# Ordinal Encoding for the hand-made classes of the target variable
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[classes])

Y = encoder.fit_transform(pd.DataFrame(Y))
Y = pd.DataFrame(Y)

In [ ]:
# XGBoost

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

model = XGBClassifier(
    num_class=len(set(Y)), 
    eval_metric='mlogloss',
    max_depth=4,
    learning_rate=0.1,
    n_estimators=90,
    random_state=42
)

model.fit(X_train, Y_train)

# predicting 
y_pred = model.predict(X_test)
print(f" Accuracy of prediction: {accuracy_score(Y_test, y_pred)}")

 Accuracy of prediction: 0.8088737201365188


In [110]:
def objective(trial):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dvalid = xgb.DMatrix(X_test, label=Y_test)

    param = {
    "objective": "multi:softprob",   # returns probabilities for each class
    "eval_metric": "mlogloss",      # multi-class log loss
    "num_class": 4,                 # total number of classes
    "verbosity": 0,
    "subsample": trial.suggest_float("subsample", 0.4, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
    "min_child_weight": trial.suggest_float("min_child_weight", 1, 10),
    }

    gbm = xgb.train(param, dtrain,
        num_boost_round=200,
        evals=[(dvalid, "validation")],
        early_stopping_rounds=20,
        verbose_eval=False,)

    preds = gbm.predict(dvalid)
    pred_labels = np.argmax(preds, axis=1)
    accuracy = accuracy_score(Y_test, pred_labels)
    return accuracy

In [111]:
study = optuna.create_study(direction="maximize", sampler=TPESampler(), pruner=optuna.pruners.MedianPruner(), study_name="Optune study object", load_if_exists=False)

[I 2025-11-04 18:25:41,785] A new study created in memory with name: Optune study object


A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object
A new study created in memory with name: Optune study object


In [112]:
study.enqueue_trial(
    {
        "subsample": 0.9,
        "colsample_bytree": 0.6,
        "min_child_weight": 2,
    }
)

study.enqueue_trial(
    {
        "subsample": 0.75,
        "colsample_bytree": 0.8,
        "min_child_weight": 5,
    }
)

import logging
import sys

# Add stream handler of stdout to show the messages to see Optuna works expectedly.
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study.optimize(objective, n_trials=100, timeout=600)

[I 2025-11-04 18:25:44,634] Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.


Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 2}. Best is trial 0 with value: 0.8225255972696246.
Trial 0 finished with value: 0.8225255972696246 and parameters: {'subsample': 0.9, 'colsample_bytree

[I 2025-11-04 18:25:44,978] Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.


Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_bytree': 0.8, 'min_child_weight': 5}. Best is trial 1 with value: 0.8361774744027304.
Trial 1 finished with value: 0.8361774744027304 and parameters: {'subsample': 0.75, 'colsample_

[I 2025-11-04 18:25:45,284] Trial 2 finished with value: 0.8088737201365188 and parameters: {'subsample': 0.7798819931870113, 'colsample_bytree': 0.9841300270650649, 'min_child_weight': 6.085411843426225}. Best is trial 1 with value: 0.8361774744027304.


...

Trial 97 finished with value: 0.8054607508532423 and parameters: {'subsample': 0.48183206324846023, 'colsample_bytree': 0.7570875830658328, 'min_child_weight': 1.8453026854571213}. Best is trial 75 with value: 0.8395904436860068.
Trial 97 finished with value: 0.8054607508532423 and parameters: {'subsample': 0.48183206324846023, 'colsample_bytree': 0.7570875830658328, 'min_child_weight': 1.8453026854571213}. Best is trial 75 with value: 0.8395904436860068.
Trial 97 finished with value: 0.8054607508532423 and parameters: {'subsample': 0.48183206324846023, 'colsample_bytree': 0.7570875830658328, 'min_child_weight': 1.8453026854571213}. Best is trial 75 with value: 0.8395904436860068.
Trial 97 finished with value: 0.8054607508532423 and parameters: {'subsample': 0.48183206324846023, 'colsample_bytree': 0.7570875830658328, 'min_child_weight': 1.8453026854571213}. Best is trial 75 with value: 0.8395904436860068.
Trial 97 finished with value: 0.8054607508532423 and parameters: {'subsample': 0

[I 2025-11-04 18:26:20,695] Trial 98 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7753693820720292, 'colsample_bytree': 0.6736730257518982, 'min_child_weight': 1.0009882781594253}. Best is trial 75 with value: 0.8395904436860068.


Trial 98 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7753693820720292, 'colsample_bytree': 0.6736730257518982, 'min_child_weight': 1.0009882781594253}. Best is trial 75 with value: 0.8395904436860068.
Trial 98 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7753693820720292, 'colsample_bytree': 0.6736730257518982, 'min_child_weight': 1.0009882781594253}. Best is trial 75 with value: 0.8395904436860068.
Trial 98 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7753693820720292, 'colsample_bytree': 0.6736730257518982, 'min_child_weight': 1.0009882781594253}. Best is trial 75 with value: 0.8395904436860068.
Trial 98 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7753693820720292, 'colsample_bytree': 0.6736730257518982, 'min_child_weight': 1.0009882781594253}. Best is trial 75 with value: 0.8395904436860068.
Trial 98 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.775

[I 2025-11-04 18:26:21,012] Trial 99 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7954637473445791, 'colsample_bytree': 0.790051145385689, 'min_child_weight': 6.473681385328207}. Best is trial 75 with value: 0.8395904436860068.


Trial 99 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7954637473445791, 'colsample_bytree': 0.790051145385689, 'min_child_weight': 6.473681385328207}. Best is trial 75 with value: 0.8395904436860068.
Trial 99 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7954637473445791, 'colsample_bytree': 0.790051145385689, 'min_child_weight': 6.473681385328207}. Best is trial 75 with value: 0.8395904436860068.
Trial 99 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7954637473445791, 'colsample_bytree': 0.790051145385689, 'min_child_weight': 6.473681385328207}. Best is trial 75 with value: 0.8395904436860068.
Trial 99 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.7954637473445791, 'colsample_bytree': 0.790051145385689, 'min_child_weight': 6.473681385328207}. Best is trial 75 with value: 0.8395904436860068.
Trial 99 finished with value: 0.8156996587030717 and parameters: {'subsample': 0.79546374734